In [2]:
%pip install -q scipy numpy quantiphy pandas control
from quantiphy import Quantity

Note: you may need to restart the kernel to use updated packages.


In [3]:
def parse_ltspice_log(filename):
	f = open(filename, 'r')
	values = {}
	for line in f:
		if ":" in line and "=" in line:
			key = line.split(":")[0]
			try:
				values[key] = float(line.split("=")[1].split()[0].replace("\n", ""))
			except ValueError:
				continue
			if (key == "phase_active"):
				values[key] = -values[key]
	if ("phase_active" in values and "phase_passive" in values):
		values["phase_total"] = values["phase_active"] + values["phase_passive"]
	return values

# Prints a given parameter with its unit and using multipliers
def print_param_dict(name: str, dict: dict) -> None:
	print(name, ":")
	keys = sorted(dict.keys())
	for key in keys:
		units = ""
		# Check if voltage
		if (key[0].lower() == "v"):
			units = "V"
		# Check if gain
		elif ("gain" in key.lower()):
			units = "V/V"
		# Check if phase
		elif ("phase" in key.lower()):
			units = "°"
		# Check if freq
		elif ("freq" in key.lower()):
			units = "Hz"	

		print("	", key + ": " + str(Quantity(dict[key])), units)



# 1- Projetar um oscilador RC por defasagem de fase para uma frequência de 1400Hz apresentado na figura 1. Com ampop LM324.

Parâmetros do circuito: 
- C: 2.5784 nF
- R: 18 kΩ
- RF: 512 kΩ

# 2- Simule o circuito projetado

## Simulação
<p align="center">
<img src="analises/assets/script/2-Simule_o_circuito_projetado.png" width="75%">
</p>

## Grafico

<p align="center">
<img src="analises/assets/script/2-Simule_o_circuito_projetado_plot.png" width="75%">
</p>

## Analise

Como é possivel observar o ganho total do sistema não é suficiente para que ocorra uma oscilação natural isto pode ser observado na simulação a seguir onde os bolocos não são realimentados

## Simulação dos blocos separados

<p align="center">
<img src="analises/assets/script/2-Simule_o_circuito_projetado_separado.png" width="75%">
</p>

## Grafico dos blocos separados

<p align="center">
<img src="analises/assets/script/2-Simule_o_circuito_projetado_plot_separado.png" width="75%">
</p>

## Medição

In [4]:
print_param_dict("Blocos separados", parse_ltspice_log("analises/1/script/2-Simule o circuito projetado_separado.log"))

Blocos separados :
	 frequency_active: 1.4k Hz
	 frequency_passive: 1.4k Hz
	 gain_active: 28.403 V/V
	 gain_passive: 34.302m V/V
	 gain_total: 974.29m V/V
	 phase_active: -182.71 °
	 phase_passive: -179.87 °
	 phase_total: -362.58 °
	 vin_active: 800m V
	 vin_passive: 22.723 V
	 vout_active: 22.723 V
	 vout_passive: 779.43m V


### Analise

O ganho total do sistema é aproximadamente 0.97 o que cria uma realimentação negativa do sinal causando o amortecimento do mesmo até 0

# 3- Caso tenha ocorrido saturação da tensão de saída do ampop, reduza o ganho do bloco amplificador o máximo que puder, para isto varie o potenciômetro.

## Simulação

<p align="center">
<img src="analises/assets/script/3-Ajuste_potenciometro.png" width="75%">
</p>

## Grafico

<p align="center">
<img src="analises/assets/script/3-Ajuste_potenciometro_plot.png" width="75%">
</p>

## Medição

In [5]:
print_param_dict("Blocos separados", parse_ltspice_log("analises/1/script/3-ajuste_potenciometro.log"))

Blocos separados :
	 frequency_active: 1.5161k Hz
	 frequency_passive: 1.5162k Hz
	 gain_active: 44.928 V/V
	 gain_passive: 22.258m V/V
	 gain_total: 1 V/V
	 phase_active: -185.27 °
	 phase_passive: -185.28 °
	 phase_total: -370.55 °
	 vin_active: 382.55m V
	 vin_passive: 17.188 V
	 vout_active: 17.188 V
	 vout_passive: 382.55m V


## Analise

Existe um desvio de 129Hz aproximadamente

# 5- Analise o funcionamento e os critérios de Brakhausen para: (Utilize os circuitos das figuras 2 e 3)

## Para a frequência projetada

### Simulação

**Circuito**

<p align="center">
<img src="analises/assets/script/5-analise_Brakhausen_1400Hz_A_B.png" width="75%">
</p>

<div align="center">
<p style="text-align:left;">
    <b> A-B </b>
    <span style="position:absolute; left:50%;">
        <b> B-A </b>
    </span>
</p>
<img src="analises/assets/script/5-analise_Brakhausen_1400Hz_A_B_zoom.png" width="45%">
<img src="analises/assets/script/5-analise_Brakhausen_1400Hz_B_A_zoom.png" width="45%"> 
</div>

### Grafico

<div align="center">
<p style="text-align:left;">
    <b> A-B </b>
    <span style="position:absolute; left:50%;">
        <b> B-A </b>
    </span>
</p>
<img src="analises/assets/script/5-analise_Brakhausen_1400Hz_A_B_plot.png" width="45%">
<img src="analises/assets/script/5-analise_Brakhausen_1400Hz_B_A_plot.png" width="45%"> 
</div>

### Medição

In [6]:
print_param_dict("Blocos A-B", parse_ltspice_log("analises/1/script/5-analise_Brakhausen_1400Hz_A_B.log"))
print_param_dict("Blocos B-A", parse_ltspice_log("analises/1/script/5-analise_Brakhausen_1400Hz_B_A.log"))

Blocos A-B :
	 frequency_active: 1.4k Hz
	 frequency_passive: 1.4k Hz
	 gain_active: 44.951 V/V
	 gain_passive: 34.136m V/V
	 gain_total: 1.5344 V/V
	 phase_active: 175.79 °
	 phase_passive: 180.16 °
	 phase_total: 355.95 °
	 vin_active: 600m V
	 vin_passive: 26.97 V
	 vout_active: 26.97 V
	 vout_passive: 920.66m V
Blocos B-A :
	 frequency_active: 1.4k Hz
	 frequency_passive: 1.4k Hz
	 gain_active: 44.935 V/V
	 gain_passive: 18.849m V/V
	 gain_total: 846.98m V/V
	 phase_active: -184.76 °
	 phase_passive: -190.15 °
	 phase_total: -374.9 °
	 vin_active: 452.37m V
	 vin_passive: 24 V
	 vout_active: 20.327 V
	 vout_passive: 452.37m V


### Analise

Nenhum dos dois circuitos cumprem os criterios de Brakhausen, quando o bloco passivo é conectado no ativo a fase é um pouco maior que 360° e o ganho é 50% acima de 1 V/V, já quando o ativo é conectado no passivo a fase é um pouco menor 360° e o ganho diminui bastante (0.84 V/V)

## Para a frequencia natural (1.5161kHz)

### Simulação

**Circuito**

<p align="center">
<img src="analises/assets/script/5-analise_Brakhausen_1516Hz_A_B.png" width="75%">
</p>

<div align="center">
<p style="text-align:left;">
    <b> A-B </b>
    <span style="position:absolute; left:50%;">
        <b> B-A </b>
    </span>
</p>
<img src="analises/assets/script/5-analise_Brakhausen_1516Hz_A_B_zoom.png" width="45%">
<img src="analises/assets/script/5-analise_Brakhausen_1516Hz_B_A_zoom.png" width="45%"> 
</div>

### Grafico

<div align="center">
<p style="text-align:left;">
    <b> A-B </b>
    <span style="position:absolute; left:50%;">
        <b> B-A </b>
    </span>
</p>
<img src="analises/assets/script/5-analise_Brakhausen_1516Hz_A_B_plot.png" width="45%">
<img src="analises/assets/script/5-analise_Brakhausen_1516Hz_B_A_plot.png" width="45%"> 
</div>

### Medição

In [7]:
print("Bloco A: Ativo")
print("Bloco B: Passivo")
print()

print_param_dict("Blocos A-B", parse_ltspice_log("analises/1/script/5-analise_Brakhausen_1516Hz_A_B.log"))
print_param_dict("Blocos B-A", parse_ltspice_log("analises/1/script/5-analise_Brakhausen_1516Hz_B_A.log"))

Bloco A: Ativo
Bloco B: Passivo

Blocos A-B :
	 frequency_active: 1.5161k Hz
	 frequency_passive: 1.5161k Hz
	 gain_active: 44.926 V/V
	 gain_passive: 40.104m V/V
	 gain_total: 1.8017 V/V
	 phase_active: -184.54 °
	 phase_passive: -175.24 °
	 phase_total: -359.78 °
	 vin_active: 600m V
	 vin_passive: 26.956 V
	 vout_active: 26.956 V
	 vout_passive: 1.081 V
Blocos B-A :
	 frequency_active: 1.516k Hz
	 frequency_passive: 1.516k Hz
	 gain_active: 44.909 V/V
	 gain_passive: 22.367m V/V
	 gain_total: 1.0045 V/V
	 phase_active: 175.01 °
	 phase_passive: 174.15 °
	 phase_total: 349.15 °
	 vin_active: 536.81m V
	 vin_passive: 24 V
	 vout_active: 24.108 V
	 vout_passive: 536.81m V


### Analise

Quando o bloco ativo é conectado no passivo para a frequencia natural o oscilador cumpre os criterios de Brakhausen para a fase (360°), porém para ganho é relativemnte maior do que necessario para cumprir o criterio isso pode explicar a leve saturação na oscilação.
Já quando o ativo é conectado no passivo existe uma desviação do criterio de Brakhausen, a causa dessa desviação vai ser abordada no seguinte item



# 6- Verifique via simulação a influência das impedâncias de entrada e saída dos blocos A e B.

Sera analisado a influencia do bloco A no bloco B porque segundo a simulação anterior o bloco A está influenciando o bloco B


## Simulação

**Circuito**

<p align="center">
<img src="analises/assets/script/6-analise_impedancia_1516Hz_B_A.png" width="75%">
</p>

<div align="center">
<p style="text-align:left;">
    <b> A-B </b>
    <span style="position:absolute; left:50%;">
        <b> B-A </b>
    </span>
</p>
<img src="analises/assets/script/6-analise_impedancia_1516Hz_B_A_zoom.png" width="45%">
<img src="analises/assets/script/6-analise_impedancia_1516Hz_B_zoom.png" width="45%"> 
</div>

## Grafico

<div align="center">
<p style="text-align:left;">
    <b> B-A </b>
    <span style="position:absolute; left:50%;">
        <b> B </b>
    </span>
</p>
<img src="analises/assets/script/6-analise_impedancia_1516Hz_B_A_plot.png" width="45%">
<img src="analises/assets/script/6-analise_impedancia_1516Hz_B_plot.png" width="45%"> 
</div>

## Medição

In [8]:
print("Bloco A: Ativo")
print("Bloco B: Passivo")
print()

print_param_dict("Blocos B-A", parse_ltspice_log("analises/1/script/6-analise_impedancia_1516Hz_B_A.log"))
print_param_dict("Blocos B", parse_ltspice_log("analises/1/script/6-analise_impedancia_1516Hz_B.log"))

Bloco A: Ativo
Bloco B: Passivo

Blocos B-A :
	 frequency_active: 1.516k Hz
	 frequency_passive: 1.516k Hz
	 gain_active: 44.909 V/V
	 gain_passive: 22.367m V/V
	 gain_total: 1.0045 V/V
	 phase_active: 175.01 °
	 phase_passive: 174.15 °
	 phase_total: 349.15 °
	 vin_active: 536.81m V
	 vin_passive: 24 V
	 vout_active: 24.108 V
	 vout_passive: 536.81m V
Blocos B :
	 frequency_passive: 1.516k Hz
	 gain_passive: 40.546m V/V
	 phase_passive: 184.65 °
	 vin_active: 0 V
	 vin_passive: 24 V
	 vout_active: 0 V
	 vout_passive: 973.09m V


## Analise

Ao colocar o bloco A na saida do bloco B existe uma dimição do ganho passivo como também uma alteração na fase, isso quer dizer que a impedancia de entrada do bloco A esta afetando o bloco B

# 7- Calcule a impedância de entrada e de saída dos blocos A e B

## Calculos

=====================>> Adicionar caluclo impedancia

## Analise

A impedancia de entrada do circuito ativo é igual a R3 e como R3 é igual a R6, na visão do circuito passivo o ultimo resistor (R6) acaba sendo o paralelo entre R3 e R6 o que é igual a $ \frac{R6}{2}$

# 8- Altere o valor de R10 para que o circuito oscile na frequência desejada. Explique

## Calculos

Como na visão do circuito passivo a impedancia do ultimo resistor é igual ao paralelo de R3 e R6 pode ser usando o dobro do valor nos dois resistores resultando para a visão do circuito passivo que R6 está no valor calculado. E o valor de Rf tem que ser recalculado para o novo valor de R3

**Novos valores**

Parâmetros do circuito: 
- C: 2.5784 nF
- R: 18 kΩ
- RD: 35.999 kΩ
- RF: 1024 kΩ


## Simulação

<p align="center">
<img src="analises/assets/script/8-novo_valor_R3.png" width="75%">
</p>

## Grafico

<p align="center">
<img src="analises/assets/script/8-novo_valor_R3_plot.png" width="75%">
</p>


## Medidas

In [9]:
print_param_dict("Alteração em R3", parse_ltspice_log("analises/1/script/8-novo_valor_R3.log"))

Alteração em R3 :
	 frequency_active: 1.3447k Hz
	 frequency_passive: 1.3447k Hz
	 gain_active: 31.163 V/V
	 gain_passive: 32.089m V/V
	 gain_total: 1 V/V
	 phase_active: 175.97 °
	 phase_passive: 175.98 °
	 phase_total: 351.95 °
	 vin_active: 226.17m V
	 vin_passive: 7.0481 V
	 vout_active: 7.0481 V
	 vout_passive: 226.17m V


## Analise

O ajuste de R3 permitiu maior precissão na frequencia de oscilação como no ganho (o potenciometro de feedback precisou de 100k em vez de 100k)

## Otimização pelas não idealidades do ampop

#### Calculos

Foi modelado o ampop como um ganho ideal com um filtro passa baixa RC

<img src="analises/assets/lm324_model.png">

Foi utilizada uma simulação de malha aberta de um lm324 para criar o modelo, os vlaores achados foram:
Lm324 model:
- R: 25.293 R
- C: 436.89 nF
- open_loop_gain: 40.099 dB

E com esse modelo foi achado um valor para R4 para chegar em uma frequencia mais proximo do desejado, o valor de R4 achado foi: 14.189k

#### Simulação

<p align="center">
<img src="analises/assets/script/8-novo_valor_R3_opt.png" width="75%">
</p>

#### Grafico


<p align="center">
<img src="analises/assets/script/8-novo_valor_R3_opt_plot.png" width="75%">
</p>


#### Medição

In [10]:
print_param_dict("Alteração em R3", parse_ltspice_log("analises/1/script/8-novo_valor_R3_opt.log"))

Alteração em R3 :
	 frequency_active: 1.4747k Hz
	 frequency_passive: 1.4747k Hz
	 gain_active: 25.068 V/V
	 gain_passive: 39.892m V/V
	 gain_total: 1 V/V
	 phase_active: -184.78 °
	 phase_passive: -184.78 °
	 phase_total: -369.56 °
	 vin_active: 1.1314 V
	 vin_passive: 28.363 V
	 vout_active: 28.363 V
	 vout_passive: 1.1314 V


#### Analise

O valor achado com a otimização aproximou mais da frequencia desejada porém ultrapasou ela, o que significa que o modelo é um pouco pessimista em relação ao ampop